In [76]:
data_path="C:\\Users\\idz\\Desktop\\ine\\data\\train"
out_path="C:\\Users\\idz\\Desktop\\ine\\data\\train\\cricket"

In [75]:
import pandas as pd
import os
import numpy as np
data=pd.read_csv(os.path.join(data_path,'bat_ball.csv'))
data.head()

,class,x-axis,y-axis,width,height,name,image_width,image_height
0,ball,308,382,26,16,U5_3_9.png,680,720
1,bat,351,202,57,26,U5_3_9.png,680,720
2,ball,235,370,27,24,U4_7_27.png,680,720
3,bat,314,337,56,85,U4_7_27.png,680,720
4,ball,238,373,24,22,U4_7_27.png,680,720


In [43]:
data['class']=np.where(data['class']=='ball',0,1)

In [44]:
data.head()

,class,x-axis,y-axis,width,height,name,image_width,image_height
0,0,308,382,26,16,U5_3_9.png,680,720
1,1,351,202,57,26,U5_3_9.png,680,720
2,0,235,370,27,24,U4_7_27.png,680,720
3,1,314,337,56,85,U4_7_27.png,680,720
4,0,238,373,24,22,U4_7_27.png,680,720


In [45]:
import os
def nn(x):
    h=[]
    for i in x:
        g=os.path.splitext(i)[0]
        h.append(g)
    return h
       

    
    
    

In [46]:
data['name_new']=nn(data['name'])

In [47]:
data.head()

,class,x-axis,y-axis,width,height,name,image_width,image_height,name_new
0,0,308,382,26,16,U5_3_9.png,680,720,U5_3_9
1,1,351,202,57,26,U5_3_9.png,680,720,U5_3_9
2,0,235,370,27,24,U4_7_27.png,680,720,U4_7_27
3,1,314,337,56,85,U4_7_27.png,680,720,U4_7_27
4,0,238,373,24,22,U4_7_27.png,680,720,U4_7_27


In [48]:
t=data[['class','x-axis','y-axis','width','height']]

In [49]:
data['bbox']= t.values.tolist()
        
        

In [50]:
data.head()

,class,x-axis,y-axis,width,height,name,image_width,image_height,name_new,bbox
0,0,308,382,26,16,U5_3_9.png,680,720,U5_3_9,"[0, 308, 382, 26, 16]"
1,1,351,202,57,26,U5_3_9.png,680,720,U5_3_9,"[1, 351, 202, 57, 26]"
2,0,235,370,27,24,U4_7_27.png,680,720,U4_7_27,"[0, 235, 370, 27, 24]"
3,1,314,337,56,85,U4_7_27.png,680,720,U4_7_27,"[1, 314, 337, 56, 85]"
4,0,238,373,24,22,U4_7_27.png,680,720,U4_7_27,"[0, 238, 373, 24, 22]"


In [51]:
new_data=data[['name_new','bbox']]

In [52]:
new_data.head()

,name_new,bbox
0,U5_3_9,"[0, 308, 382, 26, 16]"
1,U5_3_9,"[1, 351, 202, 57, 26]"
2,U4_7_27,"[0, 235, 370, 27, 24]"
3,U4_7_27,"[1, 314, 337, 56, 85]"
4,U4_7_27,"[0, 238, 373, 24, 22]"


In [53]:
df=new_data.groupby('name_new')['bbox'].apply(list).reset_index(name='bboxs')

In [54]:
df.head()

,name_new,bboxs
0,U10_0_10,"[[0, 208, 347, 16, 16], [1, 207, 257, 85, 26]]"
1,U10_0_11,"[[0, 204, 357, 20, 14], [1, 191, 268, 83, 24]]"
2,U10_0_13,"[[0, 199, 387, 22, 16], [1, 174, 294, 83, 39]]"
3,U10_0_16,"[[0, 199, 459, 16, 17], [1, 168, 359, 75, 67]]"
4,U10_0_17,"[[0, 202, 493, 11, 14], [1, 162, 374, 83, 72]]"


In [68]:
pwd

'C:\\Users\\idz\\Desktop\\ine\\data\\train'

In [66]:
from sklearn import model_selection

In [70]:
from tqdm import tqdm
import shutil

In [77]:
data_path="C:\\Users\\idz\\Desktop\\ine\\data\\train"
out_path="C:\\Users\\idz\\Desktop\\ine\\data\\train\\cricket"

# main function

In [103]:
def process_data(data,data_type='train'):
    for _,row in tqdm(data.iterrows(),total=len(data)):
        image_name=row['name_new']
        bounding_boxes=row['bboxs']
        yolo_data=[]
        for bbox in bounding_boxes:
            c=bbox[0]
            x=bbox[1]
            y=bbox[2]
            w=bbox[3]
            h=bbox[4]
            x_center = x + w / 2
            y_center= y + h / 2
            x_center /=680.0
            y_center /=720.0
            w /=680.0
            h /=720.0
            yolo_data.append([c,x_center,y_center,w,h])
        yolo_data =np.array(yolo_data) 
        np.savetxt(
            os.path.join(out_path , f"labels/{data_type}/{image_name}.txt"),
                         yolo_data,
                         fmt=['%d','%f','%f','%f','%f']
        )
        shutil.copyfile(
            os.path.join(data_path,f'images/{image_name}.png'),
            os.path.join(out_path,f'images/{data_type}/{image_name}.png')
        )   
        

**train/valid split**

In [96]:
df_train,df_valid=model_selection.train_test_split(df,test_size=0.1,random_state=42,shuffle=True)
df_train=df_train.reset_index(drop=True)
df_valid=df_valid.reset_index(drop=True)

**calling the function**

In [ ]:
process_data(df_train,data_type='train')
process_data(df_valid,data_type='validation')